# 5 - Model inference

In [ ]:
import os

path_to_ortho_predict= "/content/drive/MyDrive/NOVA_course_deep_learning/data/orthomosaics/test_data/braatan_40m_20230605_sun.tif"

path_to_tiles="/content/drive/MyDrive/NOVA_course_deep_learning/data/tiles/10m_"+str(os.path.splitext( os.path.basename(path_to_ortho_predict))[0])
path_to_model="/content/drive/MyDrive/NOVA_course_deep_learning/data/annotated_data/train/seedlingsHobol_YOLOn_img640/weights/best.pt"

### 5.1 Setuping YOLOv8 and importing other libraries, mounting Google Drive

In [ ]:
%pip install ultralytics
import ultralytics
ultralytics.checks()

In [ ]:
# other libraries

!pip install geopandas
!pip install rasterio
!pip install folium matplotlib mapclassify

# general python packages

import os, glob, shutil
from pathlib import Path
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import folium

# geospatial packages

from osgeo import gdal, ogr, osr
import geopandas as gpd
from shapely.geometry import Polygon
import rasterio as rio
path_osgeo_utils= "/usr/local/lib/python3.10/dist-packages/osgeo_utils" # defines path to gdal_retile.py

In [ ]:
# mount google drive

from google.colab import drive
drive.mount('/content/drive')

### 5.2 Writing a function to do the tiling with overlap between the tiles

In [ ]:
# Params:
#   - input_ortho_path: path to orthophoto to predict on
#   - footprints_path: path to multipolygon layer of tile footprints
#   - tile_size_m: tile size in m
#   - buffer_size_m= size of overlap area around each tile in m (used for removing boundary effects)
# input_ortho_path=
# footprints_path=

def tile_orthomosaic(input_ortho_path,footprints_path, tile_size_m, buffer_size_m):

  # Define path to data

  path_data="/content/drive/MyDrive/NOVA_course_deep_learning/data"

  # read drone acquisition footprints

  footprints= gpd.read_file(footprints_path)

  # Get ortho name

  ortho_name=os.path.splitext(os.path.basename(input_ortho_path)) [0]

  # create output dir

  output_tiles_dir=path_data+"/tiles/"+str(tile_size_m)+"m_"+ortho_name
  if not os.path.exists(output_tiles_dir):
    print("Creating output folder..."+ output_tiles_dir)
    os.makedirs(output_tiles_dir)

  # get raster metadata
  # Get pixel resolution (in meters) and tile size in pixels

  src_ds = gdal.Open(input_ortho_path)                  # reads in the orthomosaic
  _, xres, _, _, _, yres  = src_ds.GetGeoTransform()    # get pixel size in meters
  print("Ortho resolution: "+str(round(xres,4))+" m")

  # Get EPSG code

  proj = osr.SpatialReference(wkt=src_ds.GetProjection())
  EPSG_code= proj.GetAttrValue('AUTHORITY',1)
  print("EPSG code: "+str(EPSG_code))

  # get number of bands

  n_bands=src_ds.RasterCount
  print("Number of bands: "+str(n_bands))

  # Compute tile and buffer size in pixels

  tile_size_px= round(tile_size_m/abs(xres))             # calculate the tile size in pixels
  buffer_size_px= round(buffer_size_m/abs(xres))         # calculate the buffer size in pixels
  tileIndex_name=ortho_name+"_tile_index"                # define name for output tile index shapefile

  # Run gdal_retile.py using

  command_retile = "python "+path_osgeo_utils+"/gdal_retile.py -targetDir " + output_tiles_dir + " " + input_ortho_path+ " -overlap " + str(buffer_size_px) + " -ps "+str(tile_size_px) + " " + str(tile_size_px) + " -of GTiff -tileIndex "+ tileIndex_name + " -tileIndexField ID"
  print(os.popen(command_retile).read())

  # cleanup tiles

  footprint_ortho= footprints[footprints['filename']==ortho_name]
  footprint_ortho_UU= footprint_ortho.geometry.unary_union

  # Load tiles shapefile

  tiles = gpd.read_file(output_tiles_dir+ "/"+ortho_name+"_tile_index.shp")
  tiles= tiles.to_crs(EPSG_code)

  # Select all tiles that are within the boundary polygon

  tiles_in = tiles[tiles.geometry.within(footprint_ortho_UU)]

  # Select all tiles that are not within the boundary polygon

  tiles_out= tiles.loc[~tiles['ID'].isin(tiles_in['ID']) ]
  print(str(len(tiles_out))+" tiles to be deleted")

  # delete tiles that are not within the footprint

  gtiffs_delete=[output_tiles_dir+ "/"+sub  for sub in tiles_out['ID']]
  for f in gtiffs_delete:
   if os.path.exists(f):
     os.remove(f)

In [ ]:
tile_orthomosaic(input_ortho_path=path_to_ortho_predict,
                 footprints_path= "/content/drive/MyDrive/NOVA_course_deep_learning/data/map_data/drone_acquisitions.geojson",
                 tile_size_m=10,    # this could be changed
                 buffer_size_m=1)   # this could be changed

### 5.3 Predicting
Regarding the arguments, here are some that can be interesting to edit:
- `imgsz`  (default=640) image size to be fed to the model for inference. This should be the same size as it was fed to model training.
- `conf`	(default=0.25)	object confidence threshold for detection. This can be defined based on the the F1 curve in the trained model's folder


In [ ]:
!yolo predict model=$path_to_model source=$path_to_tiles imgsz=1024 conf=0.391 project=$path_to_tiles name=predict_tiles save_txt=True save_conf=True save=True line_width=1